In [1]:
import time
import pandas as pd
from tqdm import tqdm

## Simple usage

Grab an iterable of any kind and wrap it around tqdm().
Use that in a loop.

Example:

In [2]:
#Simple usage
for e in tqdm(range(1,10)):
    time.sleep(0.5)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.98it/s]


In [3]:
numbers = [1, 2, 3, 4, 5, 6, 7]
for number in tqdm(numbers):
    time.sleep(number*0.1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  2.46it/s]


## Pandas progress apply

Call tqdm.pandas()  
Use progress_apply() instead of apply

Example:

In [4]:
df = pd.DataFrame({'numbers':[1, 2, 3, 4, 5],
                   'letters': ['a', 'b', 'c', 'd', 'e']})

df

,numbers,letters
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e


In [5]:
tqdm.pandas()
df['exp_numbers'] = df.numbers.progress_apply(lambda x: x ** 2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10721.64it/s]


In [6]:
df

,numbers,letters,exp_numbers
0,1,a,1
1,2,b,4
2,3,c,9
3,4,d,16
4,5,e,25


## Multithreading

For multithreading usage, if you use list comprehension, the function should update the progress bar at the end of each call.    

For example:

In [7]:
import requests
import concurrent

In [8]:
#Create a function that will parallelized
#Add the tqdm update
def request_pokemon(number:int, pbar) -> str:
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{number}')
    pbar.update(1) #update pbar
    return response.json().get('name')

#Create the iterator
numbers= range(1, 110)

#Use tqdm with the len of iterator as total
with tqdm(total=len(numbers)) as pbar:
    #Set a threadpoool executor with 6 cores and run the function for the list of parameters
    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        res = [executor.submit(request_pokemon, number, pbar) for number in numbers] #List comprehension
        concurrent.futures.wait(res)
        pokemons_parallel = [r.result() for r in res] #Gather up the results in a list


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:02<00:00, 42.28it/s]


In [9]:
print(len(pokemons_parallel))
print(pokemons_parallel[10])

109
metapod


## Async

If you don't use list comprehension, you don't need to change the code. Just wrap the iterator with tqdm().

Example:

In [10]:
import aiohttp
import asyncio
import nest_asyncio #Do this if you want to run it on JN
nest_asyncio.apply()

pokemons_async = []

async def get_pokemon(session, url):
    async with session.get(url) as resp:
        pokemon = await resp.json()
        return pokemon.get('name')

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for number in tqdm(range(1, 151)):
            url = f'https://pokeapi.co/api/v2/pokemon/{number}'
            tasks.append(asyncio.ensure_future(get_pokemon(session, url)))
        original_pokemon = await asyncio.gather(*tasks)
        for pokemon in original_pokemon:
            pokemons_async.append(pokemon)

asyncio.run(main())

100%|██████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 156581.78it/s]


In [11]:
print(len(pokemons_async))
print(pokemons_async[10])

150
metapod
